# Setting Grafting Points

This tutorial demonstrates how to set **non-uniform initial conditions** for chain propagators. As an application, we compute the end-to-end distance of a homopolymer.

**What you'll learn:**
- Setting initial conditions for chain ends (grafting points)
- Computing physical properties like end-to-end distance

**Prerequisites:** Complete `00_QuickStart.ipynb` and `Diblock.ipynb` first.

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "1"      # Single-threaded OpenMP
os.environ["OMP_NUM_THREADS"] = "1"      # Single-threaded FFT

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from polymerfts import PropagatorSolver

## 2. Simulation Parameters

| Parameter | Value | Description |
|-----------|-------|-------------|
| $N$ | 100 | Reference chain length |
| $\Delta s$ | 0.01 | Contour step size (= $1/N$) |
| $L_x, L_y, L_z$ | $12.0 R_0$ | Box dimensions |
| $m_x, m_y, m_z$ | 64 | Grid points per dimension |
| $b_A/b$ | 1.3 | Statistical segment length |

In [ ]:
# Simulation parameters
nx = [64, 64, 64]          # Grid points
lx = [12.0, 12.0, 12.0]    # Box size
ds = 0.01                  # Contour step interval
bond_lengths = {"A": 1.3}  # Statistical segment length

## 3. Create PropagatorSolver

We use `PropagatorSolver` with `grafting_points` parameter for setting custom initial conditions.

In [ ]:
# Create PropagatorSolver
solver = PropagatorSolver(
    nx=nx, lx=lx,
    ds=ds,
    bond_lengths=bond_lengths,
    bc=["periodic"]*6,
    chain_model="continuous",
    method="pseudospectral",
    reduce_memory=False
)

## 4. Add Homopolymer with Grafting Point

In graph notation:
- Two nodes: [0, 1]
- One A-type block connecting them
- Node 0 is initialized with a custom initial condition $q_{\text{init}}({\bf r}; G)$

```
G(0)---A---A(1)
```
Where G indicates the grafting point (custom initial condition).

In [ ]:
# Define node indices
v, u = 0, 1

# Define grafting point: node 0 will be initialized with q_init["G"]
grafting_points = {v: "G"}

# Add homopolymer with grafting point
solver.add_polymer(
    volume_fraction=1.0,
    blocks=[["A", 1.0, v, u]],
    grafting_points=grafting_points
)

## 5. Set Initial Conditions and Potential Fields

The initial condition $q_{\text{init}}({\bf r}; G)$ is set to a delta function $\delta({\bf r})$ at the origin. This represents a chain end fixed at a single point.

In [ ]:
# Set delta function initial condition at the origin
dv = solver.get_dv()
q_init = {"G": np.zeros(nx)}
q_init["G"][0, 0, 0] = 1.0 / dv  # Normalize by volume element

# Visualize grafting point
plt.figure(figsize=(6, 4))
plt.title('Grafting Point: $q_{init}(r; G)$')
plt.xlabel('y')
plt.ylabel('x')
plt.imshow(q_init["G"][0, :, :], extent=(0, lx[1], 0, lx[0]), origin='lower', cmap='hot')
plt.colorbar()
plt.show()

# Zero potential field (free chain)
w_A = np.zeros(nx)

## 6. Solve Modified Diffusion Equation

The propagator $q^{0 \rightarrow 1}_0(\mathbf{r}, s)$ satisfies:

$$\frac{\partial q^{0 \rightarrow 1}_0(\mathbf{r}, s)}{\partial s} = \frac{b_A^2}{6}\nabla^2 q^{0 \rightarrow 1}_0(\mathbf{r}, s) - w_A(\mathbf{r}) q^{0 \rightarrow 1}_0(\mathbf{r}, s), \quad s \in [0, 1]$$

With initial condition:

$$q^{0 \rightarrow 1}_0(\mathbf{r}, 0) = q_{\text{init}}(\mathbf{r}; G) = \delta(\mathbf{r})$$

In [ ]:
# Compute propagators with custom initial condition
solver.compute_propagators({"A": w_A}, q_init=q_init)

## 7. Compute End-to-End Distance

For a **continuous Gaussian chain**, the mean-squared end-to-end distance scales as:
\begin{align}
\left<R^2\right>_n = b^2 n
\end{align}

where $n$ is the contour length and $b$ is the statistical segment length.

In our units (with $R_0 = b\sqrt{N}$ as the length unit):
\begin{align}
\left<R^2\right>_n = \frac{(b_A/b)^2 \cdot n}{N} = (b_A/b)^2 \cdot \frac{n}{N}
\end{align}

For $b_A/b = 1.3$, we expect $\left<R^2\right>_N \cdot N/n = 1.69$.

In [ ]:
# Create mesh for computing distances
dx = np.array(lx) / np.array(nx)
x = []

# Compute minimum distance accounting for periodic BC
for i in range(3):
    positions = np.linspace(0.0, lx[i], num=nx[i], endpoint=False)
    min_distance = np.minimum(positions, lx[i] - positions)
    x.append(min_distance)

xv, yv, zv = np.meshgrid(x[0], x[1], x[2], indexing='ij')
distance_square = np.reshape(xv**2 + yv**2 + zv**2, np.prod(nx))

# Compute <R^2> at different contour positions
N = round(1.0 / ds)
print("Computing end-to-end distance:")
print(f"Expected value: (b_A/b)^2 = {bond_lengths['A']**2:.2f}")
print("")

r2_values = []
n_values = []

for n in range(10, round(N) + 1, 10):
    # Get propagator at contour position n
    q_out = solver.get_propagator(polymer=0, v=v, u=u, step=n)
    
    # Compute <R^2> weighted by propagator
    r_squared = np.sum(q_out * distance_square) / np.sum(q_out)
    
    # Normalize by contour length
    r_squared_normalized = r_squared * N / n
    
    n_values.append(n)
    r2_values.append(r_squared_normalized)
    
    print(f"n = {n:3d}, <R^2>*N/n = {r_squared_normalized:.4f}")

print(f"\nTheoretical: {bond_lengths['A']**2:.4f}")

In [ ]:
import matplotlib.animation as animation
from IPython.display import HTML

# Create animation of propagator evolution
fig, ax = plt.subplots(figsize=(8, 6))
fig.suptitle('Chain Propagator $q^{0 \\rightarrow 1}_0$ Evolution (z=0 slice)')

im = ax.imshow(np.reshape(solver.get_propagator(polymer=0, v=v, u=u, step=0), nx)[0, :, :],
               extent=(0, lx[1], 0, lx[0]), origin='lower', cmap='hot')
ax.set(xlabel='y', ylabel='x')
title = ax.set_title('$s = 0.00$')
fig.colorbar(im, ax=ax)

def update(frame):
    data = np.reshape(solver.get_propagator(polymer=0, v=v, u=u, step=frame), nx)[0, :, :]
    im.set_array(data)
    im.set_clim(vmin=np.min(data), vmax=np.max(data))
    title.set_text(f'$s = {frame*ds:.2f}$')
    return [im, title]

anim = animation.FuncAnimation(fig, update, frames=100, interval=50, repeat=True)
plt.close(fig)

HTML(anim.to_jshtml())

## 8. Summary

In this tutorial, we learned:

1. **Grafting points**: Custom initial conditions for chain propagators using `grafting_points` parameter
2. **Custom q_init**: Pass initial conditions via `compute_propagators(w_fields, q_init=...)`
3. **End-to-end distance**: Computed from propagator moments, agrees with Gaussian chain theory

### Next Steps

- **Non-periodic boundaries**: See `NonPeriodicBC.ipynb`
- **Polymers around particles**: See `NanoParticle.ipynb`
- **Self-consistent field theory**: See `../SelfConsistentFieldTheory/`